In [58]:
from typing import Callable
import numpy as np
import math

# Things to do / Ideas
1. Create functions to test
2. Calculate the error based on the degree of the Simpson's rule
3. Graph the error (and possibly a visualization of the functions)

In [5]:
def simps_2d(f: Callable[[float], float], a: float, b:float) -> float:
    m = (a+b)/2
    return ((b-a)/6)*(f(a) + 4*f(m) + f(b))

In [6]:
f = lambda x: x**2
simps_2d(f, -1, 1)

0.6666666666666666

In [23]:
def general_simpson(f: Callable[[float], float], degree: int, a: float, b: float) -> float:
    xs = np.linspace(a, b, degree, True, dtype = np.float64)
    A: list[np.float64] = []
    for i in range(1, len(xs)):
        m = (xs[i] + xs[i-1])/2
        a = (1/6) * (f(xs[i]) + 4*f(m) + f(xs[i-1]))*(xs[i]-xs[i-1])
        A.append(a)
    return sum(A)

In [72]:
print(general_simpson(f, 2, -1, 1))
f3 = lambda x: x**3 + x**2 + x + 1
print(general_simpson(f3, 5, -1, 2))
f4 = lambda x: x**4
print(general_simpson(f4, 4, -1, 2))
sinfun = lambda x: math.sin(x) + x**3
print(general_simpson(sinfun, 2, 0, math.pi/2))
expfun = lambda x: math.exp(x)
print(general_simpson(expfun, 4, 0, 1))
print(math.e - 1)

0.6666666666666666
11.25
6.625
2.5242969248984983
1.7182891699208316
1.718281828459045
